<a id='Q0'></a>
<center> <h1> Crypto Project Module 2: Bitcoin Price Modeling Experiment </h1> </center>
<center> <h2 class="blue-text" style="color: blue"> Base Data Collector Pipeline </h2> </center>
<p style="margin-bottom:1cm;"></p>
<center><h4>ZHAW CAS DAENG Group 4, July 2021</h4></center>
<p style="margin-bottom:1cm;"></p>

<div style="background:#06065c10;border-top:0.01cm solid #53c653;border-bottom:0.01cm solid #53c653;color:#303030">
    <div style="margin-left: 0.5cm;margin-top: 0.5cm;margin-bottom: 0.5cm">
        <p><strong>Goal: </strong>Get updated data streams to build the base dataset ...</p>
        <p> <strong> Outline:</strong>
        <a id="P0" name="P0"></a>
        <ol>
            <li> <a href='#I'>Import Libraries </a> </li>
            <li> <a href='#C1'>Base Data Collection</a></li>
            <li> <a href='#C2'>Data Ingestion</a></li>
            <li> <a href='#C3'>Data Organization</a></li>
            <li> <a href='#C4'>Data Wrangling</a></li>
        </ol>
    </div>
</div>

<a id='I'></a>
## <font color='darkblue'>__Import Libraries__</font>

In [1]:
%load_ext autoreload
%reload_ext autoreload

# Import all the necessary modules

import os
import re
import pathlib
import json
import sqlalchemy as db
import requests
import yfinance as yf
from pycoingecko import CoinGeckoAPI

from datetime import datetime, date

import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

import plotly.offline as pyo
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

pyo.init_notebook_mode(connected=True)
cg = CoinGeckoAPI()

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

project_dir = pathlib.Path().absolute()
workdir = project_dir.parent


In [2]:
# Import external functions

from cr_data_loader import jprint, get_json_data, get_quandl_data, get_yfinance_data, get_datareader_data, get_poloniex_data, get_eia_data
from cr_data_manipulate import check_items, value_mapper, df_date_to_col, df_preprocess, load_to_sql, create_date_table, df_save_to_csv

<a id='C1'></a>
## <font color='darkblue'>__Data Collection__</font>

#### <font color='darkblue'>__Collect Market & Exchange data__</font>

In [3]:
# Market (Asset) Data from Coingecko.com

markets_list = cg.get_coins_markets(vs_currency='usd')
market_names = []
for item in markets_list:
    market_names.append(item['name'])
df_cg_markets = pd.DataFrame.from_dict(markets_list)
df_cg_markets.columns = df_cg_markets.columns.str.title()
df_cg_markets.head()

,Id,Symbol,Name,Image,Current_Price,Market_Cap,Market_Cap_Rank,Fully_Diluted_Valuation,Total_Volume,High_24H,...,Total_Supply,Max_Supply,Ath,Ath_Change_Percentage,Ath_Date,Atl,Atl_Change_Percentage,Atl_Date,Roi,Last_Updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,34883.00,653759402456,1,7.323204e+11,22687742780,34924.00,...,2.100000e+07,2.100000e+07,64805.00,-46.29788,2021-04-14T11:54:46.763Z,67.810000,51222.84414,2013-07-06T00:00:00.000Z,None,2021-07-03T21:39:46.935Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,2225.52,259526283608,2,NaN,19913558184,2239.95,...,NaN,NaN,4356.99,-48.91225,2021-05-12T14:41:48.623Z,0.432979,513987.12478,2015-10-20T00:00:00.000Z,"{'times': 84.38266117948953, 'currency': 'btc'...",2021-07-03T21:40:02.992Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,1.00,62659368956,3,NaN,44488787382,1.01,...,6.255588e+10,NaN,1.32,-24.29461,2018-07-24T00:00:00.000Z,0.572521,74.95505,2015-03-02T00:00:00.000Z,None,2021-07-03T21:05:26.996Z
3,binancecoin,bnb,Binance Coin,https://assets.coingecko.com/coins/images/825/...,301.06,46553784653,4,5.137384e+10,982827231,302.63,...,1.705337e+08,1.705337e+08,686.31,-56.18972,2021-05-10T07:24:17.097Z,0.039818,755023.69538,2017-10-19T00:00:00.000Z,None,2021-07-03T21:39:49.273Z
4,cardano,ada,Cardano,https://assets.coingecko.com/coins/images/975/...,1.43,45766491487,5,6.422588e+10,2578407854,1.43,...,4.500000e+10,4.500000e+10,2.45,-41.65935,2021-05-16T07:44:28.033Z,0.019253,7310.41604,2020-03-13T02:22:55.044Z,None,2021-07-03T21:39:42.849Z


In [4]:
# Exchange Data from Coingecko.com

exchanges_list = cg.get_exchanges_list()
exchange_names = []
for item in exchanges_list:
    exchange_names.append(item['name'])
df_cg_exchanges = pd.DataFrame.from_dict(exchanges_list)
df_cg_exchanges.columns = df_cg_exchanges.columns.str.title() 
df_cg_exchanges.head()

,Id,Name,Year_Established,Country,Description,Url,Image,Has_Trading_Incentive,Trust_Score,Trust_Score_Rank,Trade_Volume_24H_Btc,Trade_Volume_24H_Btc_Normalized
0,binance,Binance,2017.0,Cayman Islands,,https://www.binance.com/,https://assets.coingecko.com/markets/images/52...,False,10,1,283383.397687,283383.397687
1,gdax,Coinbase Exchange,2012.0,United States,,https://www.coinbase.com,https://assets.coingecko.com/markets/images/23...,False,10,2,31474.090464,31474.090464
2,ftx_spot,FTX,2019.0,Antigua and Barbuda,,https://ftx.com/,https://assets.coingecko.com/markets/images/45...,False,10,3,22208.661546,22208.661546
3,bitmart,BitMart,2017.0,Cayman Islands,,https://www.bitmart.com/en,https://assets.coingecko.com/markets/images/23...,False,10,4,20476.035836,20476.035836
4,kraken,Kraken,2011.0,United States,,https://r.kraken.com/Q1m9x,https://assets.coingecko.com/markets/images/29...,False,10,5,14009.080006,14009.080006


In [5]:
print(f"No. of Assests:   {len(market_names)}\n" \
      f"No. of Exchanges: {len(exchange_names)}")

No. of Assests:   100
No. of Exchanges: 100


In [6]:
print(set(df_cg_markets['Symbol'].str.upper()))

{'KLAY', 'LUSD', 'WBTC', 'DAI', 'XLM', 'SNX', 'SC', 'DASH', 'NEXO', 'CRO', 'ZIL', 'OKB', 'USDC', 'ZEN', 'AAVE', 'SUSHI', 'LINK', 'XTZ', 'AVAX', 'ETC', 'FIL', 'CETH', 'BTG', 'XSUSHI', 'NANO', 'LTC', 'LEO', 'HOT', 'NEO', 'HBTC', 'CRV', 'ZEC', 'TRX', 'VET', 'LUNA', 'BTT', 'MANA', 'DOGE', 'TFUEL', 'BNT', 'MIOTA', 'CDAI', 'XDC', 'QTUM', 'HNT', 'DGB', 'BCH', 'CUSDC', 'DOT', 'GRT', 'TITAN', 'CAKE', 'MATIC', 'WAVES', 'XRP', 'BAT', 'RUNE', 'UST', 'CEL', 'QNT', 'BTC', 'OMG', 'FTT', 'ICP', 'ONT', 'ETH', 'XEM', 'USDT', 'HT', 'PAX', 'AMP', 'EOS', 'ATOM', 'ENJ', 'SAFEMOON', 'SHIB', 'YFI', 'COMP', 'TUSD', 'BUSD', 'MDX', 'EGLD', 'UNI', 'NEAR', 'MKR', 'XMR', 'ADA', 'TEL', 'BNB', 'CHZ', 'ONE', 'ALGO', 'BSV', 'STETH', 'KSM', 'DCR', 'THETA', 'SOL', 'STX', 'HBAR'}


In [7]:
print(set(df_cg_exchanges['Id'].str.upper()))

{'BITFINEX', 'FTX_US', 'BKEX', 'DECOIN', 'EXMARKETS', 'POLONIEX', 'FTX_SPOT', 'WHITEBIT', 'COIN_METRO', 'BLOCKCHAIN_COM', 'NICE_HASH', 'BITBANK', 'CHANGELLY', 'PARIBU', 'BITHUMB', 'INDEPENDENT_RESERVE', 'WAZIRX', 'LBANK', 'INDODAX', 'KORBIT', 'BITFOREX', 'BAMBOO_RELAY', 'OKEX', 'BIGONE', 'SUSHISWAP', 'DIGIFINEX', 'NARKASA', 'EMIREX', 'BINANCE_US', 'KUCOIN', 'UNISWAP_V2', 'UPBIT', 'ETOROX', 'BITMAX', 'CRYPTO_COM', 'BW', 'MERCADO_BITCOIN', 'QUOINE', 'TOKPIE', 'HBTC', 'COINTIGER', 'BITGET', 'ZAIF', 'COINDEAL', 'KRAKEN', 'BTCMARKETS', 'MXC', 'BITKUB', 'FINEXBOX', 'BTCTURK', 'LUNO', 'CURRENCY', 'GOKU', 'PANCAKESWAP', 'GDAX', 'BITHUMB_GLOBAL', 'DEXTRADE', 'BITTREX', 'ALTERDICE', 'BITOFFER', 'OKCOIN', 'BIKI', 'GEMINI', 'BTSE', 'OCEANEX', 'EXMO', 'PROBIT', 'HOO', 'COINONE', 'QUICKSWAP', 'P2PB2B', 'STOCKS_EXCHANGE', 'COINSBIT', 'BITSO', 'AAX', 'KUNA', 'ZB', 'UNISWAP', 'HUOBI', 'BTC_ALPHA', 'PHEMEX', 'STORMGAIN', 'BITRUE', 'BITMART', 'BITCOIN_COM', 'BITFLYER', 'NOMINEX', 'HOTBIT', 'BITSTAMP', 'H

#### <font color='darkblue'>__Collect Bitcoin Data from different exchanges from Quandl__</font>

In [8]:
# Pull the Bitcoin Data from Quandl

from quandl.errors.quandl_error import NotFoundError

exchanges = df_cg_exchanges['Id'].str.upper()

df_run_all = []
    
for exchange in exchanges:
    exchange_code = f"BCHARTS/{exchange}USD"
   
    try:
        df = get_quandl_data(exchange_code)
        df['Exchange'] = exchange
        df_run_all.append(df)
        
    except NotFoundError as e:
        continue

df_quandl_btc_data = pd.concat(df_run_all)

Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BCHARTS-KRAKENUSD.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BCHARTS-BITFINEXUSD.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BCHARTS-BITSTAMPUSD.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BCHARTS-CEXUSD.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BCHARTS-P2PB2BUSD.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BCHARTS-BITFLYERUSD.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BCHARTS-OKCOINUSD.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BCHARTS-HITBTCUSD.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BCHAR

In [9]:
# Rename the columns and sort the data
df_quandl_btc_data.rename(columns={'Volume (BTC)': 'Volume_BTC', 'Volume (Currency)': 'Volume_USD'}, inplace=True)
df_quandl_btc_data.sort_index(inplace=True)
df_quandl_btc_data.tail()

,Open,High,Low,Close,Volume_BTC,Volume_USD,Weighted Price,Exchange
2021-06-14,39069.50,41028.60,38828.00,40510.50,238.339197,9.475973e+06,39758.347169,CEX
2021-06-14,39035.70,41068.90,38734.10,40530.80,9383.689293,3.741106e+08,39868.176669,KRAKEN
2021-06-15,40544.19,41341.57,39526.47,40150.98,4032.715287,1.622198e+08,40225.958780,BITSTAMP
2021-06-15,40529.10,41341.80,39461.10,40154.70,6769.385480,2.727025e+08,40284.679334,KRAKEN
2021-06-15,40510.40,41200.00,39649.80,40122.00,179.199247,7.209080e+06,40229.411989,CEX


#### <font color='darkblue'>__Collect Bitcoin Data from Poloniex__</font>

In [10]:
# Pull the Bitcoin Data from Poloniex, rename columns and sort
df_poloniex_btc_data = get_poloniex_data("USDT_BTC")
df_poloniex_btc_data.columns= df_poloniex_btc_data.columns.str.title()
df_poloniex_btc_data = df_poloniex_btc_data.rename(columns={'Volume': 'Volume_USD', 'Quotevolume': 'Volume_BTC',  
                                                     'Weightedaverage': 'Weighted Price'})

df_poloniex_btc_data['Exchange'] = 'POLONIEX'
column_names_btc=['Open', 'High', 'Low', 'Close', 'Volume_BTC', 'Volume_USD', 'Weighted Price', 'Exchange']
df_poloniex_btc_data = df_poloniex_btc_data.reindex(columns=column_names_btc)
df_poloniex_btc_data.sort_index(inplace=True)
df_poloniex_btc_data.tail()

Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\USDT_BTC.json from cache


,Open,High,Low,Close,Volume_BTC,Volume_USD,Weighted Price,Exchange
date,,,,,,,,
2021-06-12,37299.845143,37455.900963,34585.362207,35523.665700,1062.619173,3.783512e+07,35605.528515,POLONIEX
2021-06-13,35525.342200,39344.000000,34768.340803,39037.795563,1070.779201,3.927173e+07,36675.846792,POLONIEX
2021-06-14,39017.162348,41021.881360,38736.118851,40514.958787,940.136867,3.744961e+07,39834.206815,POLONIEX
2021-06-15,40515.745032,41310.475878,39529.411765,40146.496881,669.002073,2.693727e+07,40264.857687,POLONIEX
2021-06-16,40146.880857,40481.566200,38323.916827,38797.907985,824.726896,3.247659e+07,39378.604957,POLONIEX


#### <font color='darkblue'>__Append Poloniex Data to have the consoildated BTC Dataset__</font>

In [11]:
# Append the Poloniex Data to Quandl Dataset, in order to have the consolidated Data
df_btc_data = df_quandl_btc_data.append(df_poloniex_btc_data)
df_btc_data.sort_values(by=['Exchange'])
df_btc_data['Market'] = "BTC"
df_btc_data = df_btc_data.sort_index()
df_btc_data.tail()

,Open,High,Low,Close,Volume_BTC,Volume_USD,Weighted Price,Exchange,Market
2021-06-15,40510.400000,41200.000000,39649.800000,40122.000000,179.199247,7.209080e+06,40229.411989,CEX,BTC
2021-06-15,40544.190000,41341.570000,39526.470000,40150.980000,4032.715287,1.622198e+08,40225.958780,BITSTAMP,BTC
2021-06-15,40529.100000,41341.800000,39461.100000,40154.700000,6769.385480,2.727025e+08,40284.679334,KRAKEN,BTC
2021-06-15,40515.745032,41310.475878,39529.411765,40146.496881,669.002073,2.693727e+07,40264.857687,POLONIEX,BTC
2021-06-16,40146.880857,40481.566200,38323.916827,38797.907985,824.726896,3.247659e+07,39378.604957,POLONIEX,BTC


## <a id='C2'></a>
## <font color='darkblue'>__Data Ingestion__</font>

#### <font color='darkblue'>__Collect Altcoin data from Poloniex__</font>

In [12]:
# Pull the Altcoin Data from Poloniex

altcoins = df_cg_markets['Symbol'].str.upper()
df_run_altcoin_new = []

for altcoin in altcoins:
    coinpair = f"BTC_{altcoin}"
    try:
        df = get_poloniex_data(coinpair)
        df['Market'] = altcoin
        df_run_altcoin_new.append(df)
                
    except ValueError as e:
        continue

df_altcoin_poloniex = pd.concat(df_run_altcoin_new)

Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BTC_ETH.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BTC_DOGE.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BTC_XRP.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BTC_DOT.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BTC_BCH.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BTC_LTC.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BTC_LINK.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BTC_ETC.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BTC_MATIC.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\0

In [13]:
# Rename the columns and sort the Data
df_altcoin_poloniex_data = df_altcoin_poloniex.sort_index()
df_altcoin_poloniex_data.columns = df_altcoin_poloniex_data.columns.str.title()
df_altcoin_poloniex_data = df_altcoin_poloniex_data.rename(columns={'Volume': 'Volume_BTC', 'Quotevolume': 'Volume_Market',  
                                                     'Weightedaverage': 'Weighted Price'})

df_altcoin_poloniex_data['Exchange'] = 'POLONIEX'
column_names_altcoin=['Open', 'High', 'Low', 'Close', 'Volume_BTC', 'Volume_Market', 'Weighted Price', 'Exchange', 'Market']
df_altcoin_poloniex_data = df_altcoin_poloniex_data.reindex(columns=column_names_altcoin)

df_altcoin_poloniex_data.tail()

,Open,High,Low,Close,Volume_BTC,Volume_Market,Weighted Price,Exchange,Market
date,,,,,,,,,
2021-06-16,0.006924,0.007169,0.006837,0.006880,25.287762,3609.130375,0.007007,POLONIEX,XMR
2021-06-16,0.000129,0.000130,0.000126,0.000128,1.611711,12655.910124,0.000127,POLONIEX,EOS
2021-06-16,0.000018,0.000019,0.000018,0.000019,0.052792,2898.861329,0.000018,POLONIEX,MANA
2021-06-16,0.000316,0.000346,0.000315,0.000346,0.580465,1772.276521,0.000328,POLONIEX,ATOM
2021-06-16,0.000126,0.000134,0.000126,0.000134,0.012571,95.312548,0.000132,POLONIEX,OMG


#### <font color='darkblue'>__Inject Other relevant Data__</font>

In [14]:
# Get Bitcoin Transaction data from Quandl

df_cost_tran = get_quandl_data('BCHAIN/CPTRA') # Cost per transaction
df_tran_block = get_quandl_data('BCHAIN/NTRBL') # Num of transactions per block
df_num_tran_pop_excl = get_quandl_data('BCHAIN/NTREP') # Num of transactions excluding popular addresses
df_btc_tran = df_cost_tran.merge(df_tran_block['Value'], how='left', left_index=True, right_index=True, 
                                 ).merge(df_num_tran_pop_excl['Value'], how='left', left_index=True, right_index=True)
df_btc_tran = df_btc_tran.rename(columns={'Value_x': 'Cost_per_TR', 'Value_y': 'Num_TR_per_Block', 'Value': 'Num_TR_ex_pop'})
df_btc_tran['Market'] = 'BTC'
df_btc_tran.tail()


Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BCHAIN-CPTRA.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BCHAIN-NTRBL.json from cache
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\BCHAIN-NTREP.json from cache


,Cost_per_TR,Num_TR_per_Block,Num_TR_ex_pop,Market
2021-06-11,118.954072,2061.923810,214427,BTC
2021-06-12,139.888315,1687.681818,219663,BTC
2021-06-13,166.955415,1417.409091,185175,BTC
2021-06-14,148.213136,1795.674074,240108,BTC
2021-06-15,142.499415,1846.267176,239231,BTC


In [15]:
# Get Commodities, Indices, etc. from Yahoo Finance

tickers = {
    'Oil_Price': 'CL=F',
    'Gold_Price': 'GC=F',
    'SP500': '^GSPC',
    'DJI': '^DJI',
    'Nasdaq': '^IXIC',
    'Tesla': 'TSLA',
    'Vol_Index': '^VIX',
    'MSCI_Index': 'XWD.TO',
    'EUR_USD': 'EURUSD=X',
    'CNY_USD': 'CNYUSD=X',
    'JPY_USD': 'JPYUSD=X'
}

start_date = '2000-01-01'

asset_list = []

for name, ticker in tickers.items():
    df = get_yfinance_data(ticker, start_date)
    df['Asset'] = name
    df['Symbol'] = re.sub(r'\W+', '', ticker)
    df['Source'] = 'YAHOO'
    asset_list.append(df)
    
df_asset_data = pd.concat(asset_list)
df_asset_data.to_csv('df_asset.csv')
df_asset_data.tail()

[*********************100%***********************]  1 of 1 completed
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\CL_F.json from cache
[*********************100%***********************]  1 of 1 completed
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\GC_F.json from cache
[*********************100%***********************]  1 of 1 completed
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\_GSPC.json from cache
[*********************100%***********************]  1 of 1 completed
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\_DJI.json from cache
[*********************100%***********************]  1 of 1 completed
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\_IXIC.json from cache
[*********************100%***********************]  1 of 1 completed
Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources

,Open,High,Low,Close,Adj Close,Volume,Asset,Symbol,Source
2021-06-10,0.009120,0.009138,0.009109,0.009123,0.009123,0,JPY_USD,JPYUSDX,YAHOO
2021-06-11,0.009141,0.009147,0.009104,0.009140,0.009140,0,JPY_USD,JPYUSDX,YAHOO
2021-06-14,0.009112,0.009123,0.009091,0.009115,0.009115,0,JPY_USD,JPYUSDX,YAHOO
2021-06-15,0.009089,0.009092,0.009078,0.009088,0.009088,0,JPY_USD,JPYUSDX,YAHOO
2021-06-16,0.009085,0.009106,0.009080,0.009084,0.009084,0,JPY_USD,JPYUSDX,YAHOO


In [16]:
# Collect Investor sentiment Data from Quandl

aaii = get_quandl_data('AAII/AAII_SENTIMENT')
inv_sentiment = aaii[['Bullish', 'Bearish', 'Bullish 8-Week Mov Avg', 'Bull-Bear Spread']]
inv_sentiment.rename(columns={'Bullish 8-Week Mov Avg':'Bull_MA_8wk'}, inplace=True)
inv_sentiment['Bear_MA_8wk'] = inv_sentiment['Bearish'].rolling(window=8).mean()
inv_sentiment['Bull-Bear Spread MA_8_wk'] = inv_sentiment['Bull_MA_8wk'] - inv_sentiment['Bear_MA_8wk']
inv_sentiment.tail()

Loaded C:\Users\akosr\CAS_DAENG\Modul_02\crypto_project_ML\00_Data\02_Resources\AAII-AAII_SENTIMENT.json from cache


,Bullish,Bearish,Bull_MA_8wk,Bull-Bear Spread,Bear_MA_8wk,Bull-Bear Spread MA_8_wk
2021-03-25,0.509494,0.205696,0.450599,0.303798,0.255039,0.195560
2021-04-01,0.458333,0.232143,0.451054,0.226190,0.239572,0.211482
2021-04-08,0.569079,0.203947,0.457611,0.365132,0.232237,0.225374
2021-04-15,0.538206,0.245847,0.460250,0.292359,0.231234,0.229015
2021-04-22,0.526814,0.205047,0.462819,0.321767,0.227142,0.235677


In [17]:
# Upsample the data to Daily from Weekly

upsampled = inv_sentiment.resample('D')
df_inv_sentiment = upsampled.interpolate(method='linear')
df_inv_sentiment.tail(10)

,Bullish,Bearish,Bull_MA_8wk,Bull-Bear Spread,Bear_MA_8wk,Bull-Bear Spread MA_8_wk
2021-04-13,0.547027,0.233876,0.459496,0.313151,0.231521,0.227975
2021-04-14,0.542616,0.239861,0.459873,0.302755,0.231378,0.228495
2021-04-15,0.538206,0.245847,0.460250,0.292359,0.231234,0.229015
2021-04-16,0.536579,0.240018,0.460617,0.296560,0.230650,0.229967
2021-04-17,0.534951,0.234190,0.460984,0.300761,0.230065,0.230919
2021-04-18,0.533324,0.228361,0.461351,0.304962,0.229480,0.231870
2021-04-19,0.531696,0.222533,0.461718,0.309164,0.228896,0.232822
2021-04-20,0.530069,0.216704,0.462085,0.313365,0.228311,0.233774
2021-04-21,0.528441,0.210876,0.462452,0.317566,0.227727,0.234725
2021-04-22,0.526814,0.205047,0.462819,0.321767,0.227142,0.235677


In [18]:
df_inv_sentiment.isna().sum()

Bullish                     28
Bearish                     28
Bull_MA_8wk                 77
Bull-Bear Spread            28
Bear_MA_8wk                 77
Bull-Bear Spread MA_8_wk    77
dtype: int64

In [19]:
df_inv_sentiment[df_inv_sentiment['Bull-Bear Spread MA_8_wk'].isna()]

,Bullish,Bearish,Bull_MA_8wk,Bull-Bear Spread,Bear_MA_8wk,Bull-Bear Spread MA_8_wk
1987-06-26,NaN,NaN,NaN,NaN,NaN,NaN
1987-06-27,NaN,NaN,NaN,NaN,NaN,NaN
1987-06-28,NaN,NaN,NaN,NaN,NaN,NaN
1987-06-29,NaN,NaN,NaN,NaN,NaN,NaN
1987-06-30,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1987-09-06,0.442857,0.37,NaN,0.072857,NaN,NaN
1987-09-07,0.454286,0.35,NaN,0.104286,NaN,NaN
1987-09-08,0.465714,0.33,NaN,0.135714,NaN,NaN
1987-09-09,0.477143,0.31,NaN,0.167143,NaN,NaN


In [20]:
df_inv_sentiment.dropna(inplace=True)

In [21]:
# EIA Data

eia_id = 'INTL.4008-8-WORL-MMTCD.A'
df_co2_emmissions = get_eia_data(eia_id)
df_co2_emmissions.tail()

,CO2_Em
Year,
2014,35328.511561
2015,35116.199154
2016,34559.643570
2017,35360.995674
2018,36227.769473


## <a id='C3'></a>
## <font color='darkblue'>__Data Organization__</font>

#### <font color='darkblue'>__Initialize__</font>

In [22]:
# Create Calendar Table
df_dates = create_date_table()
df_dates.set_index('Date', inplace = True)
df_dates.tail()

,DateID,Week_day,Day,Month,Week,Quarter,Year
Date,,,,,,,
2099-12-27,20991227,Sunday,27,12,52,4,2099
2099-12-28,20991228,Monday,28,12,53,4,2099
2099-12-29,20991229,Tuesday,29,12,53,4,2099
2099-12-30,20991230,Wednesday,30,12,53,4,2099
2099-12-31,20991231,Thursday,31,12,53,4,2099


In [23]:
# Create copies of the imported data that are being transformed

df_btc = df_btc_data.copy()
df_altcoin = df_altcoin_poloniex_data.copy()
df_fin_asset = df_asset_data.copy()
df_transactions = df_btc_tran.copy()
df_exchanges = df_cg_exchanges.copy()
df_markets = df_cg_markets.copy()
df_sentiments = df_inv_sentiment.copy()
df_emmissions = df_co2_emmissions.copy()


#### <font color='darkblue'>__Preprocess__</font>

In [24]:
# Add the Fin Assets Exchange Data to Exchange List 

exchanges_list_cg = pd.unique(df_exchanges['Id'].str.upper())
exchanges_list_cg= np.append(exchanges_list_cg, ['YAHOO'])
exchanges_list_cg.sort()
exchanges_list_cg

print(f"{exchanges_list_cg}")

['AAX' 'ALTERDICE' 'BAMBOO_RELAY' 'BIBOX' 'BIGONE' 'BIKI' 'BINANCE'
 'BINANCE_US' 'BITBANK' 'BITCOIN_COM' 'BITFINEX' 'BITFLYER' 'BITFOREX'
 'BITGET' 'BITHUMB' 'BITHUMB_GLOBAL' 'BITKUB' 'BITMART' 'BITMAX'
 'BITOFFER' 'BITRUE' 'BITSO' 'BITSTAMP' 'BITTREX' 'BKEX' 'BLOCKCHAIN_COM'
 'BTCMARKETS' 'BTCTURK' 'BTC_ALPHA' 'BTSE' 'BW' 'CEX' 'CHANGELLY'
 'COINCHECK' 'COINDEAL' 'COINEX' 'COINONE' 'COINSBIT' 'COINTIGER'
 'COIN_METRO' 'CRYPTO_COM' 'CURRENCY' 'DECOIN' 'DEXTRADE' 'DIGIFINEX'
 'EMIREX' 'ETOROX' 'EXMARKETS' 'EXMO' 'FINEXBOX' 'FTX_SPOT' 'FTX_US'
 'GATE' 'GDAX' 'GEMINI' 'GOKU' 'HBTC' 'HITBTC' 'HOO' 'HOTBIT' 'HUOBI'
 'INDEPENDENT_RESERVE' 'INDODAX' 'KICKEX' 'KORBIT' 'KRAKEN' 'KUCOIN'
 'KUNA' 'LATOKEN' 'LBANK' 'LUNO' 'MAX_MAICOIN' 'MERCADO_BITCOIN' 'MXC'
 'NARKASA' 'NICE_HASH' 'NOMINEX' 'OCEANEX' 'OKCOIN' 'OKEX' 'P2PB2B'
 'PANCAKESWAP' 'PARIBU' 'PHEMEX' 'POLONIEX' 'PROBIT' 'QUICKSWAP' 'QUOINE'
 'STOCKS_EXCHANGE' 'STORMGAIN' 'SUSHISWAP' 'TOKPIE' 'UNISWAP' 'UNISWAP_V2'
 'UPBIT' 'WAZIRX' 'WHITE

In [25]:
# Add the Fin Assets Market data to Market List

markets_list_cg = pd.unique(df_markets['Symbol'].str.upper())
add_markets = np.unique(df_fin_asset['Symbol'])
markets_list_cg = np.append(markets_list_cg, add_markets)
markets_list_cg.sort()
markets_list_cg

print(f"{markets_list_cg}")

['AAVE' 'ADA' 'ALGO' 'AMP' 'ATOM' 'AVAX' 'BAT' 'BCH' 'BNB' 'BNT' 'BSV'
 'BTC' 'BTG' 'BTT' 'BUSD' 'CAKE' 'CDAI' 'CEL' 'CETH' 'CHZ' 'CLF' 'CNYUSDX'
 'COMP' 'CRO' 'CRV' 'CUSDC' 'DAI' 'DASH' 'DCR' 'DGB' 'DJI' 'DOGE' 'DOT'
 'EGLD' 'ENJ' 'EOS' 'ETC' 'ETH' 'EURUSDX' 'FIL' 'FTT' 'GCF' 'GRT' 'GSPC'
 'HBAR' 'HBTC' 'HNT' 'HOT' 'HT' 'ICP' 'IXIC' 'JPYUSDX' 'KLAY' 'KSM' 'LEO'
 'LINK' 'LTC' 'LUNA' 'LUSD' 'MANA' 'MATIC' 'MDX' 'MIOTA' 'MKR' 'NANO'
 'NEAR' 'NEO' 'NEXO' 'OKB' 'OMG' 'ONE' 'ONT' 'PAX' 'QNT' 'QTUM' 'RUNE'
 'SAFEMOON' 'SC' 'SHIB' 'SNX' 'SOL' 'STETH' 'STX' 'SUSHI' 'TEL' 'TFUEL'
 'THETA' 'TITAN' 'TRX' 'TSLA' 'TUSD' 'UNI' 'USDC' 'USDT' 'UST' 'VET' 'VIX'
 'WAVES' 'WBTC' 'XDC' 'XEM' 'XLM' 'XMR' 'XRP' 'XSUSHI' 'XTZ' 'XWDTO' 'YFI'
 'ZEC' 'ZEN' 'ZIL']


In [26]:
# Create ID Columns for Exchanges

exchange_mapping = {}
for exchange_id, exchange_value in enumerate(exchanges_list_cg):
    exchange_mapping.update({exchange_value:exchange_id+1})
    
value_mapper(df_exchanges, 'Id', 'ExchangeID', exchange_mapping)
df_exchanges[['Id', 'Name', 'ExchangeID']].sort_values(by='Name').tail()

,Id,Name,ExchangeID
80,xt,XT.COM,98
28,zb,ZB,101
75,zaif,Zaif,100
38,bitflyer,bitFlyer,12
71,etorox,eToroX,47


In [27]:
# Create ID Columns for Markets

market_mapping = {}
for market_id, market_value in enumerate(markets_list_cg):
    market_mapping.update({market_value:market_id+1})

value_mapper(df_markets, 'Symbol', 'MarketID', market_mapping)
df_markets[['Id', 'Symbol', 'MarketID']].sort_values(by='Id').tail()

,Id,Symbol,MarketID
99,xsushi,xsushi,105
74,yearn-finance,yfi,108
67,zcash,zec,109
89,zencash,zen,110
78,zilliqa,zil,111


In [28]:
# Map the Exchange and Market IDs to the BTC, Altcoin and Fin Asset Data

value_mapper(df_btc, 'Exchange', 'ExchangeID', exchange_mapping)
value_mapper(df_btc, 'Market', 'MarketID', market_mapping)

value_mapper(df_altcoin, 'Exchange', 'ExchangeID', exchange_mapping)
value_mapper(df_altcoin, 'Market', 'MarketID', market_mapping)

value_mapper(df_fin_asset, 'Source', 'ExchangeID', exchange_mapping)
value_mapper(df_fin_asset, 'Symbol', 'MarketID', market_mapping)

value_mapper(df_transactions, 'Market', 'MarketID', market_mapping)


,Cost_per_TR,Num_TR_per_Block,Num_TR_ex_pop,Market,MarketID
2009-01-02,0.000000,1.000000,1,BTC,12
2009-01-03,0.000000,1.000000,0,BTC,12
2009-01-04,0.000000,1.000000,0,BTC,12
2009-01-05,0.000000,1.000000,0,BTC,12
2009-01-06,0.000000,1.000000,0,BTC,12
...,...,...,...,...,...
2021-06-11,118.954072,2061.923810,214427,BTC,12
2021-06-12,139.888315,1687.681818,219663,BTC,12
2021-06-13,166.955415,1417.409091,185175,BTC,12
2021-06-14,148.213136,1795.674074,240108,BTC,12


In [29]:
## Create mapping table of the columns for Preprocessing

# Create dictionary of the dataframes
data_frames = {
    'BTC Data': df_btc,
    'Altcoin Data': df_altcoin,
    'Fin Assets Data': df_fin_asset,
    'Transaction Data': df_transactions,
    'Exchanges Data': df_exchanges,
    'Markets Data': df_markets,
    'Sentiments Data': df_sentiments,
    'Emmissions Data': df_emmissions,
    'Date Table': df_dates
}

# Organize the Dataframe column names and their mapping into a dictionary
data_frames_col_translate = {}
column_list = []
column_map = {}
for df_name, df in data_frames.items():
    column_list.append(
        [{col: col.replace(" ", "") for col in df.columns}]
    )
    for col in column_list:
        data_frames_col_translate[df_name] = col

#data_frames_col_translate

#### <font color='darkblue'>__Rearrange and Save__</font>

In [30]:
# Preprocess Data for SQL DB load

df_btc_prep = df_preprocess(df_btc, 'BTC Data', data_frames_col_translate)
df_altcoin_prep = df_preprocess(df_altcoin, 'Altcoin Data', data_frames_col_translate)
df_fin_asset_prep = df_preprocess(df_fin_asset, 'Fin Assets Data', data_frames_col_translate)
df_transactions_prep = df_preprocess(df_transactions, 'Transaction Data', data_frames_col_translate)
df_sentiments_prep = df_preprocess(df_sentiments, 'Sentiments Data', data_frames_col_translate)
df_emmissions_prep = df_preprocess(df_emmissions, 'Emmissions Data', data_frames_col_translate)
df_exchanges_prep = df_preprocess(df_exchanges, 'Exchanges Data', data_frames_col_translate)
df_markets_prep = df_preprocess(df_markets, 'Markets Data', data_frames_col_translate)
df_dates_prep = df_preprocess(df_dates, 'Date Table', data_frames_col_translate)

# Additional manual preprocessing

df_markets_prep = df_markets_prep.drop('Date', axis=1)
df_exchanges_prep = df_exchanges_prep.drop('Date', axis=1)
df_exchanges_prep['Trading_Inc'] = df_exchanges_prep['Has_Trading_Incentive'].replace({True: 1, False: 0})


In [31]:
data_frames_for_sql = {
    'BTC Data': df_btc_prep,
    'Altcoin Data': df_altcoin_prep,
    'Fin Assets Data': df_fin_asset_prep,
    'Transaction Data': df_transactions_prep,
    'Exchanges Data': df_exchanges_prep,
    'Markets Data': df_markets_prep,
    'Sentiments Data': df_sentiments_prep,
    'Emmissions Data': df_emmissions_prep,
    'Date Table': df_dates_prep
}

In [32]:
# Saving the data into csv Files for external DB load

df_save_to_csv(data_frames_for_sql)

<p> <strong>(Optional Step)</strong> The Data Organization into Database itself happens outside of the notebook: 
    <li>Create Tables ralated to each other</li>
    <li>Load the csv-files coming as output from this notebook into the tables</li>
    <li>Query the Dataset as an input for further analysis</li>

Further down we are going to demonstrate a Wrangling and Transformation process within the notebook using Python...

</p>

## <a id='C4'></a>
## <font color='darkblue'>__Data Wrangling__</font>

<h4>Simulation of an ETL-Process</h4>
<p>We are going to use recursive 
    <strong>Merge->Check->Impute->Transform/Aggregate</strong>
    process on all the relevant Datasources and put together the final dataset for EDA and Modeling</p>

### <font color='darkblue'>__BTC Dataset__</font>

#### <font color='blue'>__Merge Data__</font>

In [33]:
btc_data_prepared = df_btc_prep.merge(df_exchanges_prep[['Id', 'ExchangeID']], how='left', 
                                      left_on='ExchangeID', right_on='ExchangeID').drop(columns=['ExchangeID'])
btc_data_prepared['Exchange'] = btc_data_prepared['Id'].str.upper()
btc_data_prepared['Month'] = btc_data_prepared['Date'].dt.month_name()
btc_data_prepared['Year'] = btc_data_prepared['Date'].dt.year
btc_data_prepared = btc_data_prepared[['Open', 'High', 'Low', 'Close', 'Date', 'WeightedPrice', 'Volume_BTC', 'Exchange', 'Year', 'Month']]
btc_data_prepared.tail()

,Open,High,Low,Close,Date,WeightedPrice,Volume_BTC,Exchange,Year,Month
14678,40510.400000,41200.000000,39649.800000,40122.000000,2021-06-15,40229.411989,179.199247,CEX,2021,June
14679,40544.190000,41341.570000,39526.470000,40150.980000,2021-06-15,40225.958780,4032.715287,BITSTAMP,2021,June
14680,40529.100000,41341.800000,39461.100000,40154.700000,2021-06-15,40284.679334,6769.385480,KRAKEN,2021,June
14681,40515.745032,41310.475878,39529.411765,40146.496881,2021-06-15,40264.857687,669.002073,POLONIEX,2021,June
14682,40146.880857,40481.566200,38323.916827,38797.907985,2021-06-16,39378.604957,824.726896,POLONIEX,2021,June


#### <font color='blue'>__Check for N/As and zeros__</font>

In [34]:
btc_data_prepared.isna().sum()

Open             0
High             0
Low              0
Close            0
Date             0
WeightedPrice    0
Volume_BTC       0
Exchange         0
Year             0
Month            0
dtype: int64

In [35]:
btc_data_prepared.loc[(btc_data_prepared==0).any(axis=1)]

,Open,High,Low,Close,Date,WeightedPrice,Volume_BTC,Exchange,Year,Month
17,0.0,0.0,0.0,0.0,2011-09-30,0.0,0.0,BITSTAMP,2011,September
18,0.0,0.0,0.0,0.0,2011-10-01,0.0,0.0,BITSTAMP,2011,October
19,0.0,0.0,0.0,0.0,2011-10-02,0.0,0.0,BITSTAMP,2011,October
32,0.0,0.0,0.0,0.0,2011-10-15,0.0,0.0,BITSTAMP,2011,October
33,0.0,0.0,0.0,0.0,2011-10-16,0.0,0.0,BITSTAMP,2011,October
...,...,...,...,...,...,...,...,...,...,...
12237,0.0,0.0,0.0,0.0,2019-12-26,0.0,0.0,BITFLYER,2019,December
12242,0.0,0.0,0.0,0.0,2019-12-27,0.0,0.0,BITFLYER,2019,December
12246,0.0,0.0,0.0,0.0,2019-12-28,0.0,0.0,BITFLYER,2019,December
12252,0.0,0.0,0.0,0.0,2019-12-29,0.0,0.0,BITFLYER,2019,December


In [36]:
btc_data_prepared = btc_data_prepared.replace(0, np.nan)

In [37]:
btc_data_prepared.isna().sum()

Open             315
High             315
Low              315
Close            315
Date               0
WeightedPrice    315
Volume_BTC       316
Exchange           0
Year               0
Month              0
dtype: int64

#### <font color='blue'>__Data Imputation__</font>

In [38]:
btc_data_prepared['WeightedPrice_filled'] = btc_data_prepared.groupby(['Exchange', 'Year', 'Month'])['WeightedPrice'].transform(
    lambda grp: grp.fillna(np.mean(grp)))

btc_data_prepared['Volume_filled'] = btc_data_prepared.groupby(['Exchange', 'Year', 'Month'])['Volume_BTC'].transform(
    lambda grp: grp.fillna(np.mean(grp)))

btc_data_prepared

,Open,High,Low,Close,Date,WeightedPrice,Volume_BTC,Exchange,Year,Month,WeightedPrice_filled,Volume_filled
0,5.800000,6.000000,5.650000,5.970000,2011-09-13,5.929231,58.371382,BITSTAMP,2011,September,5.929231,58.371382
1,5.580000,5.720000,5.520000,5.530000,2011-09-14,5.590798,61.145984,BITSTAMP,2011,September,5.590798,61.145984
2,5.120000,5.240000,5.000000,5.130000,2011-09-15,5.094272,80.140795,BITSTAMP,2011,September,5.094272,80.140795
3,4.820000,4.870000,4.800000,4.850000,2011-09-16,4.854515,39.914007,BITSTAMP,2011,September,4.854515,39.914007
4,4.870000,4.870000,4.870000,4.870000,2011-09-17,4.870000,0.300000,BITSTAMP,2011,September,4.870000,0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...
14678,40510.400000,41200.000000,39649.800000,40122.000000,2021-06-15,40229.411989,179.199247,CEX,2021,June,40229.411989,179.199247
14679,40544.190000,41341.570000,39526.470000,40150.980000,2021-06-15,40225.958780,4032.715287,BITSTAMP,2021,June,40225.958780,4032.715287
14680,40529.100000,41341.800000,39461.100000,40154.700000,2021-06-15,40284.679334,6769.385480,KRAKEN,2021,June,40284.679334,6769.385480
14681,40515.745032,41310.475878,39529.411765,40146.496881,2021-06-15,40264.857687,669.002073,POLONIEX,2021,June,40264.857687,669.002073


#### <font color='blue'>__Transorm / Aggregate Data__</font>

In [39]:
btc_data_prepared = btc_data_prepared.groupby(['Date'], as_index=False).agg({'Open':'mean', 'High': 'mean', 'Low': 'mean', 'Close': 'mean',
                                                                             'WeightedPrice_filled':'mean', 
                                                                             'Volume_filled': 'sum'})

In [40]:
btc_data_prepared.isna().sum()

Date                     0
Open                    18
High                    18
Low                     18
Close                   18
WeightedPrice_filled     0
Volume_filled            0
dtype: int64

In [41]:
# Simple 30-Day Moving Average

btc_data_prepared['SMA_30'] = btc_data_prepared.loc[:,'WeightedPrice_filled'].rolling(window=30).mean().fillna(method='bfill')

In [42]:
# Exponential Moving Average

btc_data_prepared['EMA'] = btc_data_prepared.loc[:,'WeightedPrice_filled'].ewm(span=40, adjust=False).mean()

In [43]:
btc_data_prepared.set_index('Date', inplace=True)

In [44]:
## Bitcoin Dataset prepared
btc_data_prepared

,Open,High,Low,Close,WeightedPrice_filled,Volume_filled,SMA_30,EMA
Date,,,,,,,,
2011-09-13,5.800000,6.000000,5.650000,5.970000,5.929231,58.371382,4.970674,5.929231
2011-09-14,5.580000,5.720000,5.520000,5.530000,5.590798,61.145984,4.970674,5.912722
2011-09-15,5.120000,5.240000,5.000000,5.130000,5.094272,80.140795,4.970674,5.872797
2011-09-16,4.820000,4.870000,4.800000,4.850000,4.854515,39.914007,4.970674,5.823125
2011-09-17,4.870000,4.870000,4.870000,4.870000,4.870000,0.300000,4.970674,5.776631
...,...,...,...,...,...,...,...,...
2021-06-12,37306.038786,37425.437741,34674.490552,35540.706425,35703.103536,10778.020022,38387.127672,41385.904053
2021-06-13,35546.550550,39358.507500,34826.635201,39038.613891,36983.375708,11648.335645,37946.415450,41171.146573
2021-06-14,39039.470587,41042.345340,38764.154713,40522.262197,39825.577717,16843.445954,37659.088549,41105.509068


### <font color='darkblue'>__Altcoin Dataset__</font>

#### <font color='blue'>__Merge Data__</font>

In [45]:
df_altcoin_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36645 entries, 0 to 36644
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           36645 non-null  datetime64[ns]
 1   Open           36645 non-null  float64       
 2   High           36645 non-null  float64       
 3   Low            36645 non-null  float64       
 4   Close          36645 non-null  float64       
 5   Volume_BTC     36645 non-null  float64       
 6   Volume_Market  36645 non-null  float64       
 7   WeightedPrice  36645 non-null  float64       
 8   Exchange       36645 non-null  object        
 9   Market         36645 non-null  object        
 10  ExchangeID     36645 non-null  int64         
 11  MarketID       36645 non-null  int64         
dtypes: datetime64[ns](1), float64(7), int64(2), object(2)
memory usage: 3.4+ MB


In [46]:
altcoin_data_prepared = df_altcoin_prep.merge(df_markets_prep[['Symbol', 'MarketID']], how='left', 
                                      left_on='MarketID', right_on='MarketID').drop(columns=['MarketID'])
altcoin_data_prepared['Symbol'] = altcoin_data_prepared['Symbol'].str.upper()
altcoin_data_prepared['Month'] = altcoin_data_prepared['Date'].dt.month_name()
altcoin_data_prepared['Year'] = altcoin_data_prepared['Date'].dt.year
altcoin_data_prepared_copy = altcoin_data_prepared.copy()
altcoin_data_prepared = altcoin_data_prepared[['Date', 'WeightedPrice', 'Symbol', 'Year', 'Month']]
altcoin_data_prepared.tail()

,Date,WeightedPrice,Symbol,Year,Month
36640,2021-06-16,0.007007,XMR,2021,June
36641,2021-06-16,0.000127,EOS,2021,June
36642,2021-06-16,0.000018,MANA,2021,June
36643,2021-06-16,0.000328,ATOM,2021,June
36644,2021-06-16,0.000132,OMG,2021,June


#### <font color='blue'>__Check for N/As and zeros__</font>

In [47]:
altcoin_data_prepared.isna().sum()

Date             0
WeightedPrice    0
Symbol           0
Year             0
Month            0
dtype: int64

In [48]:
altcoin_data_prepared.loc[(altcoin_data_prepared==0).any(axis=1)]

,Date,WeightedPrice,Symbol,Year,Month


#### <font color='blue'>__Transorm / Aggregate Data__</font>

In [49]:
# Altcoin Prices are in BTC, we want USD, so we have to multiply them through with the Bitcoin/USD Price

altcoin_data_grouped = altcoin_data_prepared.groupby(['Date'], as_index=False).agg({'WeightedPrice':'mean'})
altcoin_data_grouped.set_index('Date', inplace=True)
altcoin_data_grouped = altcoin_data_grouped.merge(btc_data_prepared['WeightedPrice_filled'], how='left',
                                                   left_index=True, right_index=True)

altcoin_data_grouped['WeightedPrice_USD'] = altcoin_data_grouped['WeightedPrice'] * altcoin_data_grouped['WeightedPrice_filled']
altcoin_data_grouped.drop(columns=['WeightedPrice', 'WeightedPrice_filled'], inplace=True)

In [50]:
# Exponential Moving Average

altcoin_data_grouped['Altcoin_EMA'] = altcoin_data_grouped.loc[:,'WeightedPrice_USD'].ewm(span=40, adjust=False).mean()

### <font color='darkblue'>__Consolidate Bitcoin and Altcoin Data__</font>

#### <font color='blue'>__Merge Data__</font>

In [51]:
btc_altcoin_merged = btc_data_prepared.merge(altcoin_data_grouped['Altcoin_EMA'], how='left', left_index=True, right_index=True)
btc_altcoin_merged['Altcoin_EMA_filled'] = btc_altcoin_merged['Altcoin_EMA'].fillna(
    value=btc_altcoin_merged['Altcoin_EMA'].rolling(window=30, center=True, min_periods=2).mean())

#### <font color='blue'>__Check for N/As and zeros__</font>

In [52]:
btc_altcoin_merged.isna().sum()

Open                     18
High                     18
Low                      18
Close                    18
WeightedPrice_filled      0
Volume_filled             0
SMA_30                    0
EMA                       0
Altcoin_EMA             858
Altcoin_EMA_filled      845
dtype: int64

In [53]:
btc_altcoin_merged[btc_altcoin_merged['Altcoin_EMA'].isna()]

,Open,High,Low,Close,WeightedPrice_filled,Volume_filled,SMA_30,EMA,Altcoin_EMA,Altcoin_EMA_filled
Date,,,,,,,,,,
2011-09-13,5.80000,6.000000,5.6500,5.970000,5.929231,58.371382,4.970674,5.929231,NaN,NaN
2011-09-14,5.58000,5.720000,5.5200,5.530000,5.590798,61.145984,4.970674,5.912722,NaN,NaN
2011-09-15,5.12000,5.240000,5.0000,5.130000,5.094272,80.140795,4.970674,5.872797,NaN,NaN
2011-09-16,4.82000,4.870000,4.8000,4.850000,4.854515,39.914007,4.970674,5.823125,NaN,NaN
2011-09-17,4.87000,4.870000,4.8700,4.870000,4.870000,0.300000,4.970674,5.776631,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2014-01-13,841.11788,862.487498,748.9525,825.302500,823.730189,33433.937224,748.044584,766.620222,NaN,21.430505
2014-01-14,825.58000,848.145750,803.8825,820.722500,829.071415,14174.016540,747.820918,769.666621,NaN,21.191420
2014-01-15,821.91250,854.077500,816.0050,845.943410,840.437696,19520.884658,751.403488,773.118869,NaN,20.953940


In [54]:
btc_altcoin_merged.isna().sum()

Open                     18
High                     18
Low                      18
Close                    18
WeightedPrice_filled      0
Volume_filled             0
SMA_30                    0
EMA                       0
Altcoin_EMA             858
Altcoin_EMA_filled      845
dtype: int64

In [55]:
## Check for the missing values in the Altcoin EMA
btc_altcoin_merged.head(850)

,Open,High,Low,Close,WeightedPrice_filled,Volume_filled,SMA_30,EMA,Altcoin_EMA,Altcoin_EMA_filled
Date,,,,,,,,,,
2011-09-13,5.800000,6.000000,5.650000,5.970000,5.929231,58.371382,4.970674,5.929231,NaN,NaN
2011-09-14,5.580000,5.720000,5.520000,5.530000,5.590798,61.145984,4.970674,5.912722,NaN,NaN
2011-09-15,5.120000,5.240000,5.000000,5.130000,5.094272,80.140795,4.970674,5.872797,NaN,NaN
2011-09-16,4.820000,4.870000,4.800000,4.850000,4.854515,39.914007,4.970674,5.823125,NaN,NaN
2011-09-17,4.870000,4.870000,4.870000,4.870000,4.870000,0.300000,4.970674,5.776631,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2014-01-05,831.720000,948.420000,825.243333,902.953333,886.448868,45143.565295,744.967446,724.134836,NaN,22.820695
2014-01-06,914.296667,1001.386667,872.820333,916.860000,938.724029,51273.050373,752.707934,734.602601,NaN,22.837994
2014-01-07,905.202600,929.111883,787.745000,796.245000,859.850024,63828.858054,757.044961,740.712232,NaN,22.710424


#### <font color='blue'>__Data Imputation__</font>

In [56]:
## Drop observations before 2014
btc_altcoin_prepared = btc_altcoin_merged[btc_altcoin_merged.index>='2014-01-05']

In [57]:
btc_altcoin_prepared.drop('Altcoin_EMA', axis=1, inplace=True)

In [58]:
btc_altcoin_prepared.rename(columns={'WeightedPrice_filled': 'WeightedPrice', 'Volume_filled': 'Volume', 
                                     'EMA': 'EMA_40', 'Altcoin_EMA_filled': 'Altcoin_EMA_40'}, inplace=True)

In [59]:
btc_altcoin_prepared.isna().sum()

Open              0
High              0
Low               0
Close             0
WeightedPrice     0
Volume            0
SMA_30            0
EMA_40            0
Altcoin_EMA_40    0
dtype: int64

#### <font color='blue'>__Transorm / Aggregate Data__</font>

In [60]:
## Merged Bitcoin dataset with aggregate Altcoin Prices
btc_altcoin_prepared.tail()

,Open,High,Low,Close,WeightedPrice,Volume,SMA_30,EMA_40,Altcoin_EMA_40
Date,,,,,,,,,
2021-06-12,37306.038786,37425.437741,34674.490552,35540.706425,35703.103536,10778.020022,38387.127672,41385.904053,1625.680659
2021-06-13,35546.550550,39358.507500,34826.635201,39038.613891,36983.375708,11648.335645,37946.415450,41171.146573,1615.682670
2021-06-14,39039.470587,41042.345340,38764.154713,40522.262197,39825.577717,16843.445954,37659.088549,41105.509068,1611.179115
2021-06-15,40524.858758,41298.461470,39541.695441,40143.544220,40251.226947,11650.302087,37447.764410,41063.836769,1607.979634
2021-06-16,40146.880857,40481.566200,38323.916827,38797.907985,39378.604957,824.726896,37292.674537,40981.630339,1605.230198


### <font color='darkblue'>__Add the Prices of Top10 Altcoins as predictors__</font>

In [61]:
altcoin_data_prepared_copy.tail()

,Date,Open,High,Low,Close,Volume_BTC,Volume_Market,WeightedPrice,Exchange,Market,ExchangeID,Symbol,Month,Year
36640,2021-06-16,0.006924,0.007169,0.006837,0.006880,25.287762,3609.130375,0.007007,POLONIEX,XMR,85,XMR,June,2021
36641,2021-06-16,0.000129,0.000130,0.000126,0.000128,1.611711,12655.910124,0.000127,POLONIEX,EOS,85,EOS,June,2021
36642,2021-06-16,0.000018,0.000019,0.000018,0.000019,0.052792,2898.861329,0.000018,POLONIEX,MANA,85,MANA,June,2021
36643,2021-06-16,0.000316,0.000346,0.000315,0.000346,0.580465,1772.276521,0.000328,POLONIEX,ATOM,85,ATOM,June,2021
36644,2021-06-16,0.000126,0.000134,0.000126,0.000134,0.012571,95.312548,0.000132,POLONIEX,OMG,85,OMG,June,2021


In [62]:
altcoin_data_prepared_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36645 entries, 0 to 36644
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           36645 non-null  datetime64[ns]
 1   Open           36645 non-null  float64       
 2   High           36645 non-null  float64       
 3   Low            36645 non-null  float64       
 4   Close          36645 non-null  float64       
 5   Volume_BTC     36645 non-null  float64       
 6   Volume_Market  36645 non-null  float64       
 7   WeightedPrice  36645 non-null  float64       
 8   Exchange       36645 non-null  object        
 9   Market         36645 non-null  object        
 10  ExchangeID     36645 non-null  int64         
 11  Symbol         36645 non-null  object        
 12  Month          36645 non-null  object        
 13  Year           36645 non-null  int64         
dtypes: datetime64[ns](1), float64(7), int64(2), object(4)
memory usage: 4.

#### <font color='blue'>__Merge__</font>

In [63]:
altcoin_for_top10 = altcoin_data_prepared.merge(altcoin_data_prepared_copy[['Date', 'Symbol', 'Volume_BTC']], 
                                                left_on=['Date', 'Symbol'], right_on=['Date', 'Symbol'])
top10_altcoin = pd.DataFrame(altcoin_for_top10.groupby('Symbol')['Volume_BTC'].sum().nlargest(10))
top10 = top10_altcoin.reset_index()['Symbol']
altcoin_data_to_merge = altcoin_data_prepared.loc[altcoin_data_prepared['Symbol'].isin(top10)]
altcoin_data_to_merge.set_index('Date', inplace=True)
altcoin_data_to_merge = altcoin_data_to_merge.merge(btc_data_prepared['WeightedPrice_filled'], 
                                                    how='left', left_index=True, right_index=True)
altcoin_data_to_merge['WeightedPrice_USD'] = altcoin_data_to_merge['WeightedPrice'] * altcoin_data_to_merge['WeightedPrice_filled']
altcoin_data_to_pivot = altcoin_data_to_merge[['WeightedPrice_USD', 'Symbol']]
altcoin_data_to_pivot

,WeightedPrice_USD,Symbol
Date,,
2014-01-18,22.809032,LTC
2014-01-19,23.287224,LTC
2014-01-20,23.657165,LTC
2014-01-21,23.403256,LTC
2014-01-21,0.001852,DOGE
...,...,...
2021-06-16,0.849003,XRP
2021-06-16,0.014964,SC
2021-06-16,170.405400,LTC


In [64]:
altcoin_data_pivoted = altcoin_data_to_pivot.pivot_table('WeightedPrice_USD', ['Date'], 'Symbol')
altcoin_data_pivoted['Year'] = altcoin_data_pivoted.index.year
altcoin_data_pivoted['Month'] = altcoin_data_pivoted.index.month_name()
altcoin_data_pivoted

Symbol,DASH,DOGE,ETC,ETH,LTC,SC,XEM,XMR,XRP,ZEC,Year,Month
Date,,,,,,,,,,,,
2014-01-18,NaN,NaN,NaN,NaN,22.809032,NaN,NaN,NaN,NaN,NaN,2014,January
2014-01-19,NaN,NaN,NaN,NaN,23.287224,NaN,NaN,NaN,NaN,NaN,2014,January
2014-01-20,NaN,NaN,NaN,NaN,23.657165,NaN,NaN,NaN,NaN,NaN,2014,January
2014-01-21,NaN,0.001852,NaN,NaN,23.403256,NaN,NaN,NaN,NaN,NaN,2014,January
2014-01-22,NaN,0.001888,NaN,NaN,23.230723,NaN,NaN,NaN,NaN,NaN,2014,January
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-12,160.894965,0.310617,51.024019,2369.757080,159.606083,0.014638,0.156737,242.596162,0.824028,124.142190,2021,June
2021-06-13,165.427009,0.315468,52.484958,2429.823687,164.441033,0.015163,0.158659,250.080477,0.857275,127.336721,2021,June
2021-06-14,171.754574,0.326171,52.797167,2529.676490,171.993129,0.015930,0.165674,266.798714,0.887314,132.687276,2021,June


#### <font color='blue'>__Check for N/As and zeros__</font>

In [65]:
altcoin_data_to_pivot.isna().sum()

WeightedPrice_USD    0
Symbol               0
dtype: int64

In [66]:
altcoin_data_pivoted.isna().sum()

Symbol
DASH       20
DOGE        3
ETC       919
ETH       567
LTC         0
SC        584
XEM       437
XMR       121
XRP       208
ZEC      1014
Year        0
Month       0
dtype: int64

#### <font color='blue'>__Data imputation__</font>

In [67]:
# Fill the NaN with the mean price per year & month

years = list(set(altcoin_data_pivoted['Year']))
months = list(set(altcoin_data_pivoted['Month']))

for year in years:
    for month in months:
        for col in altcoin_data_pivoted.columns[:10]:
            altcoin_data_pivoted[col].fillna(altcoin_data_pivoted[col][(altcoin_data_pivoted['Year']==year) & (altcoin_data_pivoted['Month']==month)].mean(), inplace=True)
            
altcoin_data_pivoted

Symbol,DASH,DOGE,ETC,ETH,LTC,SC,XEM,XMR,XRP,ZEC,Year,Month
Date,,,,,,,,,,,,
2014-01-18,3.932714,0.000278,1.853450,4.443731,22.809032,0.000063,0.000593,0.678368,0.007777,9855.274558,2014,January
2014-01-19,3.932714,0.000278,1.853450,4.443731,23.287224,0.000063,0.000593,0.678368,0.007777,9855.274558,2014,January
2014-01-20,3.932714,0.000278,1.853450,4.443731,23.657165,0.000063,0.000593,0.678368,0.007777,9855.274558,2014,January
2014-01-21,3.932714,0.001852,1.853450,4.443731,23.403256,0.000063,0.000593,0.678368,0.007777,9855.274558,2014,January
2014-01-22,3.932714,0.001888,1.853450,4.443731,23.230723,0.000063,0.000593,0.678368,0.007777,9855.274558,2014,January
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-12,160.894965,0.310617,51.024019,2369.757080,159.606083,0.014638,0.156737,242.596162,0.824028,124.142190,2021,June
2021-06-13,165.427009,0.315468,52.484958,2429.823687,164.441033,0.015163,0.158659,250.080477,0.857275,127.336721,2021,June
2021-06-14,171.754574,0.326171,52.797167,2529.676490,171.993129,0.015930,0.165674,266.798714,0.887314,132.687276,2021,June


In [68]:
altcoin_data_pivoted.isna().sum()

Symbol
DASH     0
DOGE     0
ETC      0
ETH      0
LTC      0
SC       0
XEM      0
XMR      0
XRP      0
ZEC      0
Year     0
Month    0
dtype: int64

#### <font color='blue'>__Transorm / Aggregate Data__</font>

In [69]:
btc_dataset_merged = btc_altcoin_prepared.merge(altcoin_data_pivoted, how='left', left_index=True, right_index=True)
btc_altcoin_dataset_final = btc_dataset_merged.drop(['Year', 'Month'], axis=1)
btc_altcoin_dataset_final['Day_of_Week'] = btc_altcoin_dataset_final.index.day_name()
btc_altcoin_dataset_final

,Open,High,Low,Close,WeightedPrice,Volume,SMA_30,EMA_40,Altcoin_EMA_40,DASH,DOGE,ETC,ETH,LTC,SC,XEM,XMR,XRP,ZEC,Day_of_Week
Date,,,,,,,,,,,,,,,,,,,,
2014-01-05,831.720000,948.420000,825.243333,902.953333,886.448868,45143.565295,744.967446,724.134836,22.820695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sunday
2014-01-06,914.296667,1001.386667,872.820333,916.860000,938.724029,51273.050373,752.707934,734.602601,22.837994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Monday
2014-01-07,905.202600,929.111883,787.745000,796.245000,859.850024,63828.858054,757.044961,740.712232,22.710424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tuesday
2014-01-08,795.252500,864.570703,774.847500,826.708218,821.193860,44961.690243,755.437185,744.638165,22.529380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday
2014-01-09,827.150863,856.450000,787.597710,830.072335,822.715635,41710.023679,751.339526,748.446822,22.325458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-12,37306.038786,37425.437741,34674.490552,35540.706425,35703.103536,10778.020022,38387.127672,41385.904053,1625.680659,160.894965,0.310617,51.024019,2369.757080,159.606083,0.014638,0.156737,242.596162,0.824028,124.142190,Saturday
2021-06-13,35546.550550,39358.507500,34826.635201,39038.613891,36983.375708,11648.335645,37946.415450,41171.146573,1615.682670,165.427009,0.315468,52.484958,2429.823687,164.441033,0.015163,0.158659,250.080477,0.857275,127.336721,Sunday
2021-06-14,39039.470587,41042.345340,38764.154713,40522.262197,39825.577717,16843.445954,37659.088549,41105.509068,1611.179115,171.754574,0.326171,52.797167,2529.676490,171.993129,0.015930,0.165674,266.798714,0.887314,132.687276,Monday


In [70]:
btc_altcoin_dataset_final.isna().sum()

Open               0
High               0
Low                0
Close              0
WeightedPrice      0
Volume             0
SMA_30             0
EMA_40             0
Altcoin_EMA_40     0
DASH              13
DOGE              13
ETC               13
ETH               13
LTC               13
SC                13
XEM               13
XMR               13
XRP               13
ZEC               13
Day_of_Week        0
dtype: int64

In [71]:
btc_altcoin_dataset_final.loc[(btc_altcoin_dataset_final==0).any(axis=1)]

,Open,High,Low,Close,WeightedPrice,Volume,SMA_30,EMA_40,Altcoin_EMA_40,DASH,DOGE,ETC,ETH,LTC,SC,XEM,XMR,XRP,ZEC,Day_of_Week
Date,,,,,,,,,,,,,,,,,,,,


In [72]:
btc_altcoin_dataset_final[btc_altcoin_dataset_final['ETH'].isna()]

,Open,High,Low,Close,WeightedPrice,Volume,SMA_30,EMA_40,Altcoin_EMA_40,DASH,DOGE,ETC,ETH,LTC,SC,XEM,XMR,XRP,ZEC,Day_of_Week
Date,,,,,,,,,,,,,,,,,,,,
2014-01-05,831.720000,948.420000,825.243333,902.953333,886.448868,45143.565295,744.967446,724.134836,22.820695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sunday
2014-01-06,914.296667,1001.386667,872.820333,916.860000,938.724029,51273.050373,752.707934,734.602601,22.837994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Monday
2014-01-07,905.202600,929.111883,787.745000,796.245000,859.850024,63828.858054,757.044961,740.712232,22.710424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tuesday
2014-01-08,795.252500,864.570703,774.847500,826.708218,821.193860,44961.690243,755.437185,744.638165,22.529380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday
2014-01-09,827.150863,856.450000,787.597710,830.072335,822.715635,41710.023679,751.339526,748.446822,22.325458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thursday
2014-01-10,830.097500,863.062640,805.472500,856.067640,833.495287,21696.391146,749.502752,752.595527,22.111447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Friday
2014-01-11,854.522500,902.078677,851.956385,887.560263,878.247287,27330.550462,750.066988,758.724881,21.892140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saturday
2014-01-12,898.350285,904.209973,824.245003,843.672500,860.540672,23944.994394,749.278233,763.691505,21.667996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sunday
2014-01-13,841.117880,862.487498,748.952500,825.302500,823.730189,33433.937224,748.044584,766.620222,21.430505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Monday


In [73]:
btc_altcoin_dataset_cleansed = btc_altcoin_dataset_final.dropna()

In [74]:
btc_altcoin_dataset_cleansed.isna().sum()

Open              0
High              0
Low               0
Close             0
WeightedPrice     0
Volume            0
SMA_30            0
EMA_40            0
Altcoin_EMA_40    0
DASH              0
DOGE              0
ETC               0
ETH               0
LTC               0
SC                0
XEM               0
XMR               0
XRP               0
ZEC               0
Day_of_Week       0
dtype: int64

In [75]:
btc_altcoin_dataset_cleansed.head()

,Open,High,Low,Close,WeightedPrice,Volume,SMA_30,EMA_40,Altcoin_EMA_40,DASH,DOGE,ETC,ETH,LTC,SC,XEM,XMR,XRP,ZEC,Day_of_Week
Date,,,,,,,,,,,,,,,,,,,,
2014-01-18,805.943845,825.650397,801.195185,813.720000,813.474062,9203.856192,772.041350,779.105001,22.809032,3.932714,0.000278,1.85345,4.443731,22.809032,0.000063,0.000593,0.678368,0.007777,9855.274558,Saturday
2014-01-19,813.880000,845.962500,802.705000,839.435000,822.870110,15651.503558,777.592671,781.239884,22.832358,3.932714,0.000278,1.85345,4.443731,23.287224,0.000063,0.000593,0.678368,0.007777,9855.274558,Sunday
2014-01-20,842.949510,849.072500,821.055000,833.446473,835.942243,11975.316447,785.348449,783.908292,22.872593,3.932714,0.000278,1.85345,4.443731,23.657165,0.000063,0.000593,0.678368,0.007777,9855.274558,Monday
2014-01-21,831.520000,837.679998,817.122500,827.370000,826.970163,10513.698009,791.822038,786.008871,22.327713,3.932714,0.001852,1.85345,4.443731,23.403256,0.000063,0.000593,0.678368,0.007777,9855.274558,Tuesday
2014-01-22,826.172500,828.975360,809.100000,823.004997,820.873594,11156.353993,797.920394,787.709589,21.805205,3.932714,0.001888,1.85345,4.443731,23.230723,0.000063,0.000593,0.678368,0.007777,9855.274558,Wednesday


### <font color='darkblue'>__Add the Injected data (Fin Assets, Sentiment data) as predictors and create the final dataset__</font>

#### <font color='blue'>__Merge__</font>

In [76]:
# Create a mapping dictionary and preprocess (subset, rename columns) the datasets

dct_features_inject = {
    'Fin Assets Data': [{
        'AdjClose': 'AdjClose',
        'Symbol': 'Symbol'}],
    'Transaction Data': [{
        'Cost_per_TR': 'Cost_per_TR',
        'Num_TR_per_Block': 'Num_TR_per_Block'}],
    'Sentiments Data': [{
        'Bull-BearSpreadMA_8_wk': 'Bu_Be_Spread_MA8'}]
}

df_to_preprocess_assets = df_fin_asset_prep.set_index('Date')
df_assets_to_merge = df_preprocess(df_to_preprocess_assets, 'Fin Assets Data', dct_features_inject)

df_to_preprocess_tr = df_transactions_prep.set_index('Date')
df_transactions_to_merge = df_preprocess(df_to_preprocess_tr, 'Transaction Data', dct_features_inject)

df_to_preprocess_sent = df_sentiments_prep.set_index('Date')
df_sentiments_to_merge = df_preprocess(df_to_preprocess_sent, 'Sentiments Data', dct_features_inject)

In [77]:
#Check the Headers

print(f"{df_assets_to_merge.tail()}\n" \
      f"{df_transactions_to_merge.tail()}\n" \
      f"{df_sentiments_to_merge.tail()}")


            Date  AdjClose   Symbol
52659 2021-06-10  0.009123  JPYUSDX
52660 2021-06-11  0.009140  JPYUSDX
52661 2021-06-14  0.009115  JPYUSDX
52662 2021-06-15  0.009088  JPYUSDX
52663 2021-06-16  0.009084  JPYUSDX
           Date  Cost_per_TR  Num_TR_per_Block
4543 2021-06-11   118.954072       2061.923810
4544 2021-06-12   139.888315       1687.681818
4545 2021-06-13   166.955415       1417.409091
4546 2021-06-14   148.213136       1795.674074
4547 2021-06-15   142.499415       1846.267176
            Date  Bu_Be_Spread_MA8
12273 2021-04-18          0.231870
12274 2021-04-19          0.232822
12275 2021-04-20          0.233774
12276 2021-04-21          0.234725
12277 2021-04-22          0.235677


In [78]:
# Fin Assets Dataset: Pivot the ticker prices after each other

df_assets_merge_prep = df_assets_to_merge.pivot_table('AdjClose', ['Date'], 'Symbol')
df_assets_merge_prep = df_assets_merge_prep[df_assets_merge_prep.index.year>=2014]
df_assets_merge_prep.head()

Symbol,CLF,CNYUSDX,DJI,EURUSDX,GCF,GSPC,IXIC,JPYUSDX,TSLA,VIX,XWDTO
Date,,,,,,,,,,,
2014-01-01,NaN,0.165453,NaN,1.374495,NaN,NaN,NaN,0.009497,NaN,NaN,NaN
2014-01-02,95.440002,0.165453,16441.349609,1.376671,1225.000000,1831.979980,4143.069824,0.009495,30.020000,14.23,28.320234
2014-01-03,93.959999,0.165544,16469.990234,1.366662,1238.400024,1831.369995,4131.910156,0.009539,29.912001,13.76,28.364527
2014-01-06,93.430000,0.165522,16425.099609,1.359601,1237.800049,1826.770020,4113.680176,0.009532,29.400000,13.55,28.364527
2014-01-07,93.669998,0.165489,16530.939453,1.363196,1229.400024,1837.880005,4153.180176,0.009596,29.872000,12.92,28.851738


#### <font color='blue'>__Check for N/As and zeros__</font>

In [79]:
df_assets_merge_prep.isna().sum()

Symbol
CLF        84
CNYUSDX    24
DJI        69
EURUSDX    23
GCF        85
GSPC       69
IXIC       69
JPYUSDX    23
TSLA       69
VIX        69
XWDTO      74
dtype: int64

In [80]:
df_assets_merge_prep[df_assets_merge_prep['GCF'].isna()]

Symbol,CLF,CNYUSDX,DJI,EURUSDX,GCF,GSPC,IXIC,JPYUSDX,TSLA,VIX,XWDTO
Date,,,,,,,,,,,
2014-01-01,NaN,0.165453,NaN,1.374495,NaN,NaN,NaN,0.009497,NaN,NaN,NaN
2014-01-20,NaN,0.165555,NaN,1.352759,NaN,NaN,NaN,0.009593,NaN,NaN,29.480682
2014-02-17,NaN,0.165101,NaN,1.369994,NaN,NaN,NaN,0.009846,NaN,NaN,NaN
2014-04-18,NaN,0.161054,NaN,1.381216,NaN,NaN,NaN,0.009762,NaN,NaN,NaN
2014-05-26,NaN,0.160601,NaN,1.362881,NaN,NaN,NaN,0.009808,NaN,NaN,30.251358
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-01,NaN,0.153099,NaN,1.218027,NaN,NaN,NaN,0.009687,NaN,NaN,NaN
2021-01-18,NaN,0.154328,NaN,1.206811,NaN,NaN,NaN,0.009624,NaN,NaN,61.950001
2021-02-15,NaN,0.154866,NaN,1.212209,NaN,NaN,NaN,0.009518,NaN,NaN,NaN


In [81]:
df_assets_merge_prep.loc[(df_assets_merge_prep==0).any(axis=1)]

Symbol,CLF,CNYUSDX,DJI,EURUSDX,GCF,GSPC,IXIC,JPYUSDX,TSLA,VIX,XWDTO
Date,,,,,,,,,,,


#### <font color='blue'>__Data imputation__</font>

In [82]:
df_assets_merge_prep = df_assets_merge_prep.fillna(method='bfill')

In [83]:
df_assets_merge_prep.isna().sum()

Symbol
CLF        0
CNYUSDX    0
DJI        0
EURUSDX    0
GCF        0
GSPC       0
IXIC       0
JPYUSDX    0
TSLA       0
VIX        0
XWDTO      0
dtype: int64

#### <font color='blue'>__Transform / Aggregate Data__</font>

In [84]:
df_tran_merge = df_transactions_to_merge.set_index('Date')
df_sent_merge = df_sentiments_to_merge.set_index('Date')

df_new_features = df_assets_merge_prep.merge(df_tran_merge, how='left', left_index=True, right_index=True).merge(df_sent_merge, how='left', left_index=True, right_index=True)

In [85]:
df_new_features.tail()

,CLF,CNYUSDX,DJI,EURUSDX,GCF,GSPC,IXIC,JPYUSDX,TSLA,VIX,XWDTO,Cost_per_TR,Num_TR_per_Block,Bu_Be_Spread_MA8
Date,,,,,,,,,,,,,,
2021-06-10,70.290001,0.156580,34466.238281,1.218200,1894.199951,4239.180176,14020.330078,0.009123,610.119995,16.100000,65.599998,114.596509,2143.923729,NaN
2021-06-11,70.910004,0.156438,34479.601562,1.217256,1877.400024,4247.439941,14069.419922,0.009140,609.890015,15.650000,66.029999,118.954072,2061.923810,NaN
2021-06-14,70.879997,0.156306,34393.750000,1.211020,1864.000000,4255.149902,14174.139648,0.009115,617.690002,16.389999,66.169998,148.213136,1795.674074,NaN
2021-06-15,72.120003,0.156318,34299.328125,1.212224,1854.500000,4246.589844,14072.860352,0.009088,599.359985,17.020000,66.360001,142.499415,1846.267176,NaN
2021-06-16,71.570000,0.156128,34033.671875,1.212709,1813.300049,4223.700195,14039.683594,0.009084,604.869995,18.150000,66.360001,NaN,NaN,NaN


In [86]:
df_new_features.isna().sum()

CLF                  0
CNYUSDX              0
DJI                  0
EURUSDX              0
GCF                  0
GSPC                 0
IXIC                 0
JPYUSDX              0
TSLA                 0
VIX                  0
XWDTO                0
Cost_per_TR          1
Num_TR_per_Block     1
Bu_Be_Spread_MA8    39
dtype: int64

In [87]:
df_new_features.fillna(method='ffill', inplace=True)

#### <font color='darkred'>__Consolidate Final Dataset__</font>

In [88]:
btc_full_dataset = btc_altcoin_dataset_cleansed.merge(df_new_features, how='left', left_index=True, right_index=True)

In [89]:
btc_full_dataset.isna().sum()

Open                  0
High                  0
Low                   0
Close                 0
WeightedPrice         0
Volume                0
SMA_30                0
EMA_40                0
Altcoin_EMA_40        0
DASH                  0
DOGE                  0
ETC                   0
ETH                   0
LTC                   0
SC                    0
XEM                   0
XMR                   0
XRP                   0
ZEC                   0
Day_of_Week           0
CLF                 774
CNYUSDX             774
DJI                 774
EURUSDX             774
GCF                 774
GSPC                774
IXIC                774
JPYUSDX             774
TSLA                774
VIX                 774
XWDTO               774
Cost_per_TR         774
Num_TR_per_Block    774
Bu_Be_Spread_MA8    774
dtype: int64

In [90]:
df_nas = btc_full_dataset[btc_full_dataset['CLF'].isna()]
df_nas

,Open,High,Low,Close,WeightedPrice,Volume,SMA_30,EMA_40,Altcoin_EMA_40,DASH,...,GCF,GSPC,IXIC,JPYUSDX,TSLA,VIX,XWDTO,Cost_per_TR,Num_TR_per_Block,Bu_Be_Spread_MA8
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-18,805.943845,825.650397,801.195185,813.720000,813.474062,9203.856192,772.041350,779.105001,22.809032,3.932714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-19,813.880000,845.962500,802.705000,839.435000,822.870110,15651.503558,777.592671,781.239884,22.832358,3.932714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-25,786.562340,817.762500,781.466195,807.052500,802.904204,13267.051134,810.532415,790.102275,20.356988,3.932714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-26,808.527500,838.702950,800.625000,819.235347,822.441459,17230.700253,813.804297,791.679797,19.874845,3.932714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-02-01,806.635685,831.140000,803.573152,818.085000,819.619379,13004.605092,826.531205,793.914345,17.205693,3.932714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-30,34640.627864,36447.775650,33505.200000,35684.246035,35359.126273,8503.900158,47038.779111,46189.594334,1810.423646,178.820649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-05,36859.908715,37909.984400,34893.383024,35530.994549,36324.274380,10754.773985,43092.364278,43807.805163,1722.627065,188.839005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-06,35531.982049,36458.383832,35295.364493,35798.607008,35947.639411,6386.451460,42391.120118,43424.382443,1708.360963,188.844094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
# Check if the missing observations are weekends...

df_nas_last_10 = df_nas[df_nas.columns[-15 :]]
df_nas_last_10.replace(np.nan, 1, inplace=True)
df_nas_last_10.groupby('Day_of_Week').count()

,CLF,CNYUSDX,DJI,EURUSDX,GCF,GSPC,IXIC,JPYUSDX,TSLA,VIX,XWDTO,Cost_per_TR,Num_TR_per_Block,Bu_Be_Spread_MA8
Day_of_Week,,,,,,,,,,,,,,
Saturday,387,387,387,387,387,387,387,387,387,387,387,387,387,387
Sunday,387,387,387,387,387,387,387,387,387,387,387,387,387,387


In [92]:
# use forward fill to impute the data

btc_full_dataset_cleansed = btc_full_dataset.fillna(method='ffill')

In [93]:
btc_full_dataset_cleansed.dropna(inplace=True)

In [94]:
btc_full_dataset_cleansed.tail()

,Open,High,Low,Close,WeightedPrice,Volume,SMA_30,EMA_40,Altcoin_EMA_40,DASH,...,GCF,GSPC,IXIC,JPYUSDX,TSLA,VIX,XWDTO,Cost_per_TR,Num_TR_per_Block,Bu_Be_Spread_MA8
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-12,37306.038786,37425.437741,34674.490552,35540.706425,35703.103536,10778.020022,38387.127672,41385.904053,1625.680659,160.894965,...,1877.400024,4247.439941,14069.419922,0.009140,609.890015,15.650000,66.029999,118.954072,2061.923810,0.235677
2021-06-13,35546.550550,39358.507500,34826.635201,39038.613891,36983.375708,11648.335645,37946.415450,41171.146573,1615.682670,165.427009,...,1877.400024,4247.439941,14069.419922,0.009140,609.890015,15.650000,66.029999,118.954072,2061.923810,0.235677
2021-06-14,39039.470587,41042.345340,38764.154713,40522.262197,39825.577717,16843.445954,37659.088549,41105.509068,1611.179115,171.754574,...,1864.000000,4255.149902,14174.139648,0.009115,617.690002,16.389999,66.169998,148.213136,1795.674074,0.235677
2021-06-15,40524.858758,41298.461470,39541.695441,40143.544220,40251.226947,11650.302087,37447.764410,41063.836769,1607.979634,174.072066,...,1854.500000,4246.589844,14072.860352,0.009088,599.359985,17.020000,66.360001,142.499415,1846.267176,0.235677
2021-06-16,40146.880857,40481.566200,38323.916827,38797.907985,39378.604957,824.726896,37292.674537,40981.630339,1605.230198,168.991708,...,1813.300049,4223.700195,14039.683594,0.009084,604.869995,18.150000,66.360001,142.499415,1846.267176,0.235677


In [95]:
btc_full_dataset_cleansed.isna().sum()

Open                0
High                0
Low                 0
Close               0
WeightedPrice       0
Volume              0
SMA_30              0
EMA_40              0
Altcoin_EMA_40      0
DASH                0
DOGE                0
ETC                 0
ETH                 0
LTC                 0
SC                  0
XEM                 0
XMR                 0
XRP                 0
ZEC                 0
Day_of_Week         0
CLF                 0
CNYUSDX             0
DJI                 0
EURUSDX             0
GCF                 0
GSPC                0
IXIC                0
JPYUSDX             0
TSLA                0
VIX                 0
XWDTO               0
Cost_per_TR         0
Num_TR_per_Block    0
Bu_Be_Spread_MA8    0
dtype: int64

#### <font color='darkred'>__Save Final Dataset__</font>

In [96]:
data_path = f"{workdir}\\"
btc_full_dataset_cleansed.to_csv(data_path+'btc_base_dataset_NEW.csv')

In [97]:
df_check = pd.read_csv(data_path+'btc_base_dataset_NEW.csv')
df_check.tail()

,Date,Open,High,Low,Close,WeightedPrice,Volume,SMA_30,EMA_40,Altcoin_EMA_40,...,GCF,GSPC,IXIC,JPYUSDX,TSLA,VIX,XWDTO,Cost_per_TR,Num_TR_per_Block,Bu_Be_Spread_MA8
2700,2021-06-12,37306.038786,37425.437741,34674.490552,35540.706425,35703.103536,10778.020022,38387.127672,41385.904053,1625.680659,...,1877.400024,4247.439941,14069.419922,0.009140,609.890015,15.650000,66.029999,118.954072,2061.923810,0.235677
2701,2021-06-13,35546.550550,39358.507500,34826.635201,39038.613891,36983.375708,11648.335645,37946.415450,41171.146573,1615.682670,...,1877.400024,4247.439941,14069.419922,0.009140,609.890015,15.650000,66.029999,118.954072,2061.923810,0.235677
2702,2021-06-14,39039.470587,41042.345340,38764.154713,40522.262197,39825.577717,16843.445954,37659.088549,41105.509068,1611.179115,...,1864.000000,4255.149902,14174.139648,0.009115,617.690002,16.389999,66.169998,148.213136,1795.674074,0.235677
2703,2021-06-15,40524.858758,41298.461470,39541.695441,40143.544220,40251.226947,11650.302087,37447.764410,41063.836769,1607.979634,...,1854.500000,4246.589844,14072.860352,0.009088,599.359985,17.020000,66.360001,142.499415,1846.267176,0.235677
2704,2021-06-16,40146.880857,40481.566200,38323.916827,38797.907985,39378.604957,824.726896,37292.674537,40981.630339,1605.230198,...,1813.300049,4223.700195,14039.683594,0.009084,604.869995,18.150000,66.360001,142.499415,1846.267176,0.235677
